In [59]:
#Cleaning the data
import pandas as pd

In [60]:
excel_path = "/content/drive/MyDrive/assignment/Analytics Position Case Study.xlsx"
xls = pd.ExcelFile(excel_path)

In [61]:
# clean each sheet
gameplay_data = xls.parse('User Gameplay data', skiprows=3)
gameplay_data.columns = ['User ID', 'Games Played', 'Datetime']
gameplay_data.dropna(inplace=True) #remove rows or columns containing missing values
gameplay_data['User ID'] = gameplay_data['User ID'].astype(int)
gameplay_data['Games Played'] = gameplay_data['Games Played'].astype(int)
gameplay_data['Datetime'] = pd.to_datetime(gameplay_data['Datetime'])

In [62]:
deposit_data = xls.parse('Deposit Data', skiprows=3)
deposit_data.columns = ['User ID', 'Datetime', 'Amount']
deposit_data.dropna(inplace=True) #remove rows or columns containing missing values
deposit_data['User ID'] = deposit_data['User ID'].astype(int)
deposit_data['Amount'] = deposit_data['Amount'].astype(float)
deposit_data['Datetime'] = pd.to_datetime(deposit_data['Datetime'])

In [63]:
withdrawal_data = xls.parse('Withdrawal Data', skiprows=3)
withdrawal_data.columns = ['User ID', 'Datetime', 'Amount']
withdrawal_data.dropna(inplace=True)
withdrawal_data['User ID'] = withdrawal_data['User ID'].astype(int)
withdrawal_data['Amount'] = withdrawal_data['Amount'].astype(float)
withdrawal_data['Datetime'] = pd.to_datetime(withdrawal_data['Datetime'])

In [64]:
#Defining slots
def slots(data):
    hour = data.hour
    return 'S1' if hour < 12 else 'S2'

In [65]:
# Add slot columns to each dataset
gameplay_data['Slot'] = gameplay_data['Datetime'].apply(slots)
deposit_data['Slot'] = deposit_data['Datetime'].apply(slots)
withdrawal_data['Slot'] = withdrawal_data['Datetime'].apply(slots)

In [66]:
gameplay_data.head()

,User ID,Games Played,Datetime,Slot
0,851,1,2022-01-10,S1
1,717,1,2022-01-10,S1
2,456,1,2022-01-10,S1
3,424,1,2022-01-10,S1
4,845,1,2022-01-10,S1


In [67]:
deposit_data.head()

,User ID,Datetime,Amount,Slot
0,357,2022-01-10 00:03:00,2000.0,S1
1,776,2022-01-10 00:03:00,2500.0,S1
2,492,2022-01-10 00:06:00,5000.0,S1
3,803,2022-01-10 00:07:00,5000.0,S1
4,875,2022-01-10 00:09:00,1500.0,S1


In [68]:
withdrawal_data.head()

,User ID,Datetime,Amount,Slot
0,190,2022-01-10 00:03:00,5872.0,S1
1,159,2022-01-10 00:16:00,9540.0,S1
2,164,2022-01-10 00:24:00,815.0,S1
3,946,2022-01-10 00:29:00,23000.0,S1
4,763,2022-01-10 00:40:00,9473.0,S1


In [69]:
# Count deposits and withdrawals per user
deposit_counts = deposit_data.groupby(['User ID','Slot']).size().reset_index(name='Deposit Count')
withdrawal_counts = withdrawal_data.groupby(['User ID','Slot']).size().reset_index(name='Withdrawal Count')

In [70]:
# Merge the counts
transaction_counts = pd.merge(deposit_counts, withdrawal_counts, on=['User ID','Slot'], how='outer')
transaction_counts.fillna(0, inplace=True)

In [71]:
#How many more times did a player do deposit than withdrawal
transaction_counts['Diff'] = transaction_counts['Deposit Count'] - transaction_counts['Withdrawal Count']
transaction_counts['Deposit than withdrawal point'] = transaction_counts['Diff'].apply(lambda x: 0.001 * max(x, 0))

In [72]:
# Aggregate total games played per user
total_games_played = gameplay_data.groupby(['User ID','Slot'])['Games Played'].sum().reset_index()
#Number of games played points
total_games_played['Game Play point'] = 0.2 * total_games_played['Games Played']

In [73]:
# Aggregate totals of deposit and withdrawal
deposit_total = deposit_data.groupby(['User ID', 'Slot'])['Amount'].sum().reset_index(name='Total Deposit')
withdrawal_total = withdrawal_data.groupby(['User ID','Slot'])['Amount'].sum().reset_index(name='Total Withdrawal')

In [74]:
#combine all to use in loyalty points formula
loyalty_df = pd.merge(deposit_total, withdrawal_total, on=['User ID', 'Slot'], how='outer')
loyalty_df = pd.merge(loyalty_df, deposit_counts, on=['User ID', 'Slot'], how='outer')
loyalty_df = pd.merge(loyalty_df, withdrawal_counts, on=['User ID', 'Slot'], how='outer')
loyalty_df = pd.merge(loyalty_df, total_games_played, on=['User ID', 'Slot'], how='outer')
loyalty_df.fillna(0, inplace=True)

In [75]:
# Apply final loyalty point formula
loyalty_df['Loyalty Point'] = (
    0.01 * loyalty_df['Total Deposit'] +
    0.005 * loyalty_df['Total Withdrawal'] +
    0.001 * loyalty_df.apply(lambda row: max(row['Deposit Count'] - row['Withdrawal Count'], 0), axis=1) +
    0.2 * loyalty_df['Games Played']
)

In [76]:
# Show top users by loyalty points
loyalty_df_sorted = loyalty_df.sort_values(by='Loyalty Point', ascending=False).head(10)
loyalty_df_sorted[['User ID', 'Slot','Loyalty Point']]

,User ID,Slot,Loyalty Point
1262,634,S1,45576.045
1263,634,S2,38267.280
196,99,S1,14798.356
1338,672,S2,13733.220
422,212,S2,13224.361
1127,566,S1,11267.634
1472,740,S1,9756.550
1473,740,S2,9455.274
1420,714,S1,9353.616
1337,672,S1,9024.560


## **1. Find Playerwise Loyalty points earned by Players in the following slots:-**

In [99]:
# Function to calculate loyalty points for a specific date and slot
def loyalty_by_target(target_date, target_slot):
    target_date = pd.to_datetime(target_date).normalize()

    # Filter each dataset by date and slot
    gameplay_fltr = gameplay_data[(gameplay_data['Datetime'].dt.normalize() == target_date) & (gameplay_data['Slot'] == target_slot)]
    deposit_fltr = deposit_data[(deposit_data['Datetime'].dt.normalize() == target_date) & (deposit_data['Slot'] == target_slot)]
    withdrawal_fltr = withdrawal_data[(withdrawal_data['Datetime'].dt.normalize() == target_date) & (withdrawal_data['Slot'] == target_slot)]

    #summary statistics for each user within the specified date and slot
    gameplay_summary=gameplay_fltr.groupby('User ID')['Games Played'].sum().reset_index(name='Games Played')
    deposit_amnt_summary=deposit_fltr.groupby('User ID')['Amount'].sum().reset_index(name='Total Deposit')
    withdrawal_amnt_summary=withdrawal_fltr.groupby('User ID')['Amount'].sum().reset_index(name='Total Withdrawal')
    deposit_count_summary=deposit_fltr.groupby('User ID').size().reset_index(name='Deposit Count')
    withdrawal_count_summary=withdrawal_fltr.groupby('User ID').size().reset_index(name='Withdrawal Count')

    # Merge all
    target_df = [deposit_amnt_summary, withdrawal_amnt_summary, deposit_count_summary, withdrawal_count_summary, gameplay_summary]
    result = target_df[0]
    for df in target_df[1:]:
        result = pd.merge(result, df, on='User ID', how='outer')
    result.fillna(0, inplace=True)

    # Loyalty Point formula
    result['Loyalty Point'] = (
        0.01 * result['Total Deposit'] +
        0.005 * result['Total Withdrawal'] +
        0.001 * result.apply(lambda row: max(row['Deposit Count'] - row['Withdrawal Count'], 0), axis=1) +
        0.2 * result['Games Played']
    )

    # Add context
    result['Date'] = target_date
    result['Slot'] = target_slot

    return result[['User ID', 'Date', 'Slot', 'Loyalty Point']].sort_values(by='Loyalty Point', ascending=False)


In [100]:
# Define target date-slot combinations
target_slots = [
    {'date': '2022-10-02', 'slot': 'S1'},
    {'date': '2022-10-16', 'slot': 'S2'},
    {'date': '2022-10-18', 'slot': 'S1'},
    {'date': '2022-10-26', 'slot': 'S2'},
]

In [110]:
# Calculate and display results individually
result_1 = loyalty_by_target('2022-10-02', 'S1')
result_2 = loyalty_by_target('2022-10-16', 'S2')
result_3 = loyalty_by_target('2022-10-18', 'S1')
result_4 = loyalty_by_target('2022-10-26', 'S2')

In [119]:
# Print each result clearly
print("Loyalty Points for 2nd October 2022 Slot S1")
result_1

Loyalty Points for 2nd October 2022 Slot S1


,User ID,Date,Slot,Loyalty Point


In [120]:
print("\nLoyalty Points for 16th October 2022 Slot S2")
result_2


Loyalty Points for 16th October 2022 Slot S2


,User ID,Date,Slot,Loyalty Point
365,634,2022-10-16,S2,1491.555
122,212,2022-10-16,S2,999.991
51,99,2022-10-16,S2,980.002
15,28,2022-10-16,S2,900.004
328,566,2022-10-16,S2,880.203
...,...,...,...,...
184,332,2022-10-16,S2,0.200
19,36,2022-10-16,S2,0.200
557,943,2022-10-16,S2,0.200
555,938,2022-10-16,S2,0.200


In [121]:
print("\nLoyalty Points for 18th October 2022 Slot S1")
result_3


Loyalty Points for 18th October 2022 Slot S1


,User ID,Date,Slot,Loyalty Point
391,634,2022-10-18,S1,2723.100
122,208,2022-10-18,S1,1701.401
419,673,2022-10-18,S1,900.801
92,162,2022-10-18,S1,770.000
147,245,2022-10-18,S1,750.000
...,...,...,...,...
590,943,2022-10-18,S1,0.200
598,955,2022-10-18,S1,0.200
600,957,2022-10-18,S1,0.200
602,965,2022-10-18,S1,0.200


In [122]:
print("\nLoyalty Points for 26th October 2022 Slot S2")
result_4


Loyalty Points for 26th October 2022 Slot S2


,User ID,Date,Slot,Loyalty Point
447,714,2022-10-26,S2,2000.001
227,369,2022-10-26,S2,1501.915
393,634,2022-10-26,S2,1237.010
329,538,2022-10-26,S2,1200.403
0,2,2022-10-26,S2,900.002
...,...,...,...,...
581,928,2022-10-26,S2,0.200
624,995,2022-10-26,S2,0.200
10,15,2022-10-26,S2,0.200
2,6,2022-10-26,S2,0.200


In [116]:
# Merge all four result tables into a single DataFrame
final_loyalty_by_slot = pd.concat([
    result_1,
    result_2,
    result_3,
    result_4
]).reset_index(drop=True)

# Display the final merged result
print("Loyalty Points for All Specified Slots:")
final_loyalty_by_slot

Loyalty Points for All Specified Slots:


,User ID,Date,Slot,Loyalty Point
0,634,2022-10-16,S2,1491.555
1,212,2022-10-16,S2,999.991
2,99,2022-10-16,S2,980.002
3,28,2022-10-16,S2,900.004
4,566,2022-10-16,S2,880.203
...,...,...,...,...
1839,928,2022-10-26,S2,0.200
1840,995,2022-10-26,S2,0.200
1841,15,2022-10-26,S2,0.200
1842,6,2022-10-26,S2,0.200


** Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October.
In case of tie, number of games played should be taken as the next criteria for ranking.**

In [149]:
# Filter data for the month of October
oct_start = pd.to_datetime("2022-10-01")
oct_end = pd.to_datetime("2022-10-31")

In [150]:
# Filter October data for gameplay, deposits, withdrawals
oct_gameplay = gameplay_data[
    (gameplay_data['Datetime'] >= oct_start) & (gameplay_data['Datetime'] <= oct_end)
]
oct_deposit = deposit_data[
    (deposit_data['Datetime'] >= oct_start) & (deposit_data['Datetime'] <= oct_end)
]
oct_withdrawal = withdrawal_data[
    (withdrawal_data['Datetime'] >= oct_start) & (withdrawal_data['Datetime'] <= oct_end)
]


In [151]:
# Group and summarize data per user
oct_gameplay_summary = oct_gameplay.groupby('User ID')['Games Played'].sum().reset_index(name='Total Games Played')
oct_deposit_amt = oct_deposit.groupby('User ID')['Amount'].sum().reset_index(name='Total Deposit')
oct_withdrawal_amt = oct_withdrawal.groupby('User ID')['Amount'].sum().reset_index(name='Total Withdrawal')
oct_deposit_count = oct_deposit.groupby('User ID').size().reset_index(name='Deposit Count')
oct_withdrawal_count = oct_withdrawal.groupby('User ID').size().reset_index(name='Withdrawal Count')

In [152]:
# Merge all October summaries
oct_data = [oct_deposit_amt, oct_withdrawal_amt, oct_deposit_count, oct_withdrawal_count, oct_gameplay_summary]
oct_result = oct_data[0]
for df in oct_data[1:]:
    oct_result = pd.merge(oct_result, df, on='User ID', how='outer')
oct_result.fillna(0, inplace=True)

In [153]:
# Calculate loyalty points
oct_result['Loyalty Point'] = (
    0.01 * oct_result['Total Deposit'] +
    0.005 * oct_result['Total Withdrawal'] +
    0.001 * oct_result.apply(lambda row: max(row['Deposit Count'] - row['Withdrawal Count'], 0), axis=1) +
    0.2 * oct_result['Total Games Played']
)

In [158]:
# Display final ranked table
oct_result_sorted = oct_result.sort_values(by=['Loyalty Point', 'Total Games Played'], ascending=[False, False]).reset_index(drop=True)
# Assign rank based on sorted position
oct_result_sorted['Rank'] = oct_result_sorted.index + 1

In [160]:
oct_result_sorted[['Rank','User ID', 'Loyalty Point', 'Total Games Played']]

,Rank,User ID,Loyalty Point,Total Games Played
0,1,634,58121.295,21.0
1,2,714,14180.823,4.0
2,3,212,13447.226,0.0
3,4,672,12338.422,7.0
4,5,99,11989.946,4.0
...,...,...,...,...
991,992,388,0.200,1.0
992,993,507,0.200,1.0
993,994,546,0.200,1.0
994,995,661,0.200,1.0


**What is the average deposit amount?**

In [161]:
# Calculate the average deposit amount for the entire dataset
average_deposit_amount = deposit_data['Amount'].mean()
average_deposit_amount

np.float64(5492.185399701801)

**What is the average deposit amount per user in a month?**

In [162]:
# Extract month from the datetime
deposit_data['Month'] = deposit_data['Datetime'].dt.to_period('M')

In [163]:
# Group by User ID and Month, then calculate monthly deposit average per user
monthly_user_deposit = deposit_data.groupby(['User ID', 'Month'])['Amount'].sum().reset_index()

In [164]:
# alculate average deposit amount per user per month
average_deposit_per_user_per_month = monthly_user_deposit['Amount'].mean()
average_deposit_per_user_per_month

np.float64(22026.846596136154)

**What is the average number of games played per user?**

In [165]:
# Group total games played per user
games_played_per_user = gameplay_data.groupby('User ID')['Games Played'].sum().reset_index()

In [166]:
# Calculate the average number of games played per user
average_games_played_per_user = games_played_per_user['Games Played'].mean()
average_games_played_per_user

np.float64(355.267)